In [15]:
# hybrid model DLS

# P 05
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.activations import relu
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_blobs
import numpy as np
import random 


#tf.enable_eager_execution()


class LSLayer(tf.keras.layers.Layer):
    
    def __init__(self,  num_outputs_s, num_outputs_r, num_outputs_l, activation=sigmoid, wstd = 0.3, bstd = 0.5):
        super(LSLayer, self).__init__()
        self.num_outputs_l = num_outputs_l
        self.num_outputs_s = num_outputs_s 
        self.num_outputs_r = num_outputs_r
        self.num_outputs = num_outputs_l + num_outputs_s + num_outputs_r
        self.activation = activation
        self.wstd = wstd
        self.bstd = bstd
        
    def build(self, input_shape):  
        self.num_inputs = input_shape[-1]
        self.kernel = self.add_weight("kernel",
                                      shape=(int(input_shape[-1]),
                                             self.num_outputs), 
                                      initializer=tf.keras.initializers.RandomNormal(stddev=self.wstd),
                                     trainable=True)

        self.bias = self.add_weight("bias",
                                      shape=[self.num_outputs],
                                    initializer=tf.keras.initializers.RandomNormal(stddev=self.bstd),
                                   trainable=True)
        
        params = self.get_weights()
    
        for j in range(self.num_outputs):
            r = 0
            for i in range(self.num_inputs):
                params[0][i][j] = params[0][i][j]*random.random()*3
                r += (params[0][i][j]-0.5)**2
            r = math.sqrt(r)
            params[1][j] = r

        self.set_weights(params )
        
    
    # F2 method LS layer
    def call(self, input):
        
        isp = input.shape
        In1 = tf.transpose(input)
        kernel_S, kernel_L  = tf.split(self.kernel,[ self.num_outputs_s + self.num_outputs_r, self.num_outputs_l ], axis = 1 )
        bias_S, bias_L  = tf.split(self.bias,[ self.num_outputs_s +  self.num_outputs_r, self.num_outputs_l ], axis = 0 )
        
        # case spherical
        
        s_shape  = self.num_outputs_s + self.num_outputs_r
        In2 = tf.stack([In1] * s_shape)
        InD = tf.transpose(In2)
        WD = tf.stack([kernel_S] * isp[0])
        ddd = WD - InD
        dd0 = tf.math.multiply(ddd, ddd)
        dd1 = tf.math.reduce_sum(dd0, axis =1)
        dd2 = tf.cast(dd1,tf.double)
        dd3 = tf.sqrt(dd2)
        d_r = tf.cast(dd3,tf.float32)
        d_R = tf.abs(bias_S)
        d_rR = tf.math.divide_no_nan(d_r,d_R)
        d_x0 = tf.ones(d_rR.shape) - d_rR
        result_S = tf.math.scalar_mul(6,d_x0)
        result_S = sigmoid(result_S)
        
        # case linear

        d_1 = tf.stack([bias_L] * isp[0])
        result_L = tf.matmul(input, kernel_L) + d_1 
        result_L = relu(result_L)

        #case empty, merge
        
        '''
        #print (self.num_outputs_r)
        if self.num_outputs_r > 0:
            r_S, _ = tf.split (result_S,[self.num_outputs_s, self.num_outputs_r],axis=1 )
            r_1 = np.zeros((result_S.shape[0],self.num_outputs_r))
            result_R = tf.cast(tf.constant(r_1),tf.float32)
            result = tf.concat([r_S, result_R, result_L],axis=1)            
            #print (self.num_outputs_s, self.num_outputs_r)
            #print ("result_S", result_S)
            #print ("result_L", result_L)
            #print ("result", result)
        else:
            result = tf.concat([result_S, result_L],axis=1)        
        '''
        
        result = tf.concat([result_S, result_L],axis=1)        
        
        return result
    

class NN_Model(Model):
    
    def __init__(self,c,hs,hr,hl):
        super(NN_Model, self).__init__()
        self.d1 = LSLayer(hs,hr,hl)
        self.d2 = Dense(c)
        
    def call(self, x):
        x = self.d1(x)
        #print ("call benn:",x, tf.math.reduce_sum(x))
        return self.d2(x)
        
        


#@tf.function
def train_step(datas, labels,modelk,loss_objectk,optimizerk,train_lossk,train_accuracyk):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = modelk(datas, training=True)
        loss = loss_objectk(labels, predictions)
    gradients = tape.gradient(loss, modelk.trainable_variables)
    optimizerk.apply_gradients(zip(gradients, modelk.trainable_variables))

    train_lossk(loss)
    train_accuracyk(labels, predictions)

#@tf.function
def test_step(datas, labels,modelk,loss_objectk,test_lossk,test_accuracyk):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    
    predictions = modelk(datas, training=False)
    t_loss = loss_objectk(labels, predictions)

    test_lossk(t_loss)
    test_accuracyk(labels, predictions)
    

def analyse_NN (model, train_ds,optimizer):
    
    for datas, labels in train_ds:
        dA = datas.numpy()
        dL = labels.numpy()
        for i in range(2):
            tA1 = tf.constant(dA[i])
            tA = tf.reshape(tA1,[1,dA.shape[1]])
            predictions = model(tA, training=False)
            
            print (dA[i], predictions.numpy(),dL[i])
            to1 = model.d1 (tA)
            print (to1.numpy())
            to2 = model.d2 (to1)
            print (to2.numpy())
            print ("-----------")

    
def update_NN_model (model, train_ds,optimizer):

    rdb = 0
    odb = 0
    #N = min(5, model.d1.num_outputs_r)   # number of new SSN nodes
    N = model.d1.num_outputs_r   # number of new SSN nodes
    
    if N <= 0:
        return
    
    # k-means
    
    baditems = []
    for datas, labels in train_ds:
        predictions = model(datas, training=False)
        for i in range(datas.shape[0]):
            #print (datas.numpy()[i], predictions.numpy()[i], np.argmax(predictions.numpy()[i]), labels.numpy()[i],np.argmax(labels.numpy()[i]))
            if np.argmax(predictions.numpy()[i]) != np.argmax(labels.numpy()[i]):
                rdb = rdb + 1
                baditems.append(datas.numpy()[i])
            odb = odb + 1        
    #print ("pontossag:",(odb-rdb)/odb, len(baditems))
    N = min(N, len(baditems))
    if N == 0:
        return
    inds = random.sample(range(len(baditems)), N)
    
    print ("update layer")
    #print (baditems)
    centers = KMeans(n_clusters=N).fit(baditems).cluster_centers_
    #print ("centers:")
    #print (centers)
    neww = np.zeros((model.d1.num_inputs,N))
    for i in range(N):
        for j in range(model.d1.num_inputs):
            neww[j,i] = centers[i][j]
    #print ("neww")
    #print (neww)
            
    xu = model.d1.get_weights()
        
    idx = random.sample(range(model.d1.num_outputs_s + model.d1.num_outputs_r),N)
    for j1 in range(N):
        j = idx[j1]
        for i in range(xu[0].shape[0]):
            xu[0][i][j] = neww[i,j1]
           
    for j1 in range(N):
        j = idx[j1]
        r = 0
        for i in range(xu[0].shape[0]):
            r += (xu[0][i][j]-0.5)**2
        r = math.sqrt(r)
        xu[1][j] = r
            
            
    model.d1.set_weights(xu )
    
    #model.d1.num_outputs_s = model.d1.num_outputs_s + N
    #model.d1.num_outputs_r = model.d1.num_outputs_r - N

    #optimizer = tf.keras.optimizers.Adam
    #for var in optimizer.variables():
    #    var.assign(tf.zeros_like(var))
    

K = 10

# (x_train,y_train,x_test,y_test) = gen_data_array_cv(K)

M = x_train[0].shape[1]
C = y_train[0].shape[1]

EPOCHS = 80
Eupd = [20]

H = M * 5
HS = M
HR = M
HL = 5*M
B = 32

model = []
loss_object =  []
optimizer = []
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
val_loss = []
val_accuracy = []
train_ds = []
val_ds = []
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(B)



best_v = 0
best_t = 0



for k in range(K):
    print ("k=",k, "------------------------------")
    # Create an instance of the model
    model.append( NN_Model(C,HS,HR,HL))
    #model.append( NN_Model(H,C,0,0))

    loss_object.append(tf.keras.losses.CategoricalCrossentropy(from_logits=True))
    #loss_object = tf.keras.losses.CategoricalCrossentropy()

    optimizer.append(tf.keras.optimizers.Adam())
    train_loss.append(tf.keras.metrics.Mean(name='train_loss'))
    train_accuracy.append(tf.keras.metrics.CategoricalAccuracy(name='train_accuracy'))

    test_loss.append(tf.keras.metrics.Mean(name='test_loss'))
    test_accuracy.append(tf.keras.metrics.CategoricalAccuracy(name='test_accuracy'))
    
    val_loss.append(tf.keras.metrics.Mean(name='val_loss'))
    val_accuracy.append(tf.keras.metrics.CategoricalAccuracy(name='val_accuracy'))
    

    #print (x_train[:2])
    train_ds.append( tf.data.Dataset.from_tensor_slices(
        (x_train[k], y_train[k])).batch(B))
    val_ds.append( tf.data.Dataset.from_tensor_slices(
        (x_val[k], y_val[k])).batch(B))
    #print (train_ds)




    X = []
    Y = []
    for epoch in range(EPOCHS):
      # Reset the metrics at the start of the next epoch

        train_loss[k].reset_states()
        train_accuracy[k].reset_states()
        test_loss[k].reset_states()
        test_accuracy[k].reset_states()

        for datas, labels in train_ds[k]:
            train_step(datas, labels,model[k],loss_object[k],optimizer[k],train_loss[k],train_accuracy[k])


        for test_datas, test_labels in test_ds:
            #tpredictions = model[k](test_datas, training=False)
            test_step(test_datas, test_labels,model[k],loss_object[k],test_loss[k],test_accuracy[k])

        for val_datas, val_labels in val_ds[k]:
            #vpredictions = model[k](val_datas, training=False)
            test_step(val_datas, val_labels,model[k],loss_object[k],val_loss[k],val_accuracy[k])
            
            
        if epoch == 5:
            analyse_NN (model[k], train_ds[k], optimizer[k])
    
        if epoch in Eupd and k < K/2:
            update_NN_model (model[k], train_ds[k], optimizer[k])


        X.append(epoch)
        Y.append(test_accuracy[k].result() * 100)
        if epoch % 20 == 0:
            #print(
            #    f'Epoch {epoch + 1}, '
            #    f'Loss: {train_loss[k].result()}, '
            #    f'Val Accuracy: {val_accuracy[k].result() * 100}, '
            #    f'Test Accuracy: {test_accuracy[k].result() * 100}'
            #  )    
            
            if val_accuracy[k].result() > best_v:
                best_v = val_accuracy[k].result().numpy()
                best_t = test_accuracy[k].result().numpy()
                print("XX", best_v,best_t)
            #print(model.d1.bias.numpy())


    #print(
    #    f'Epoch {epoch + 1}, '
    #    f'Loss: {train_loss[k].result()}, '
    #    f'Val Accuracy: {val_accuracy[k].result() * 100}, '
    #    f'Test Accuracy: {test_accuracy[k].result() * 100}'
    #  )    
    #acclist.append(test_accuracy[k].result())
    print ("accuracy:",best_t)
    #plt.plot(X, Y,label="Accuracy curve")


k= 0 ------------------------------
XX 0.3188406 0.47368422
[0.1764706  0.49748743 0.6557377  0.11111111 0.07565012 0.2876304
 0.08795901 0.15      ] [[1.575108  0.7671416]] [1. 0.]
[[0.8153558  0.5839426  0.5767977  0.6374625  0.8666568  0.41474405
  0.76020616 0.6358555  0.6171553  0.6306263  0.6792692  0.5004312
  0.71275735 0.48701566 0.5668278  0.8682275  2.2207947  2.5843768
  1.7812259  1.3558127  1.828123   2.5618439  1.8417774  1.8029422
  1.2477342  2.3064904  2.834803   2.2609591  1.9420176  2.4444392
  1.5486091  2.2821596  1.3504708  1.5863352  1.6766706  2.5334852
  1.54096    1.9653355  1.9022169  2.2191048  1.9843372  2.5908024
  1.9675932  2.370582   2.2832959  2.3257012  2.0202944  1.7419354
  2.9928591  1.304437   1.7579466  2.0637932  1.4514133  1.2470984
  2.2099564  3.054634  ]]
[[1.575108  0.7671416]]
-----------
[0.1764706  0.79396987 0.52459013 0.13131313 0.4574468  0.46497765
 0.09265585 0.05      ] [[1.2618501  0.53175485]] [1. 0.]
[[0.57177395 0.42101055 0.3

update layer
XX 0.66045547 0.67105263


KeyboardInterrupt: 

In [4]:
# read benchmark data
from pandas import  read_csv
import numpy as np
import random


def gen_data_array_cv(K):
    
    
    dataT = read_csv ("pima-indians-diabetes.csv", header =None)
    dataA = np.array(dataT)    
        
    N = dataA.shape[0]
    M = dataA.shape[1]-1

    Nte = int(0.1*N)
    Ntr = N - Nte

    cval = dict()
    for i in range(N):
        cval[dataA[i][-1]] = 1
    C = len(cval.keys())
    
    minv = [0 for _ in range(M)]
    maxv = [0 for _ in range(M)]
    
    for j in range(M):
        minv[j] =  dataA[0,j]
        maxv[j] =  dataA[0,j]
        
    for i in range(N):
        for j in range(M):
            minv[j] = min( dataA[i,j] , minv[j])
            maxv[j] = max( dataA[i,j] , maxv[j])
          
    orders = random.sample(range(N), N)
    
    x2_test = np.zeros((Nte,M),dtype='float32')
    y2_test = np.zeros((Nte,C))
    
    ite = 0
    for n in range(Ntr, N):
        i = orders[n]
        for j in range(M):
            x2_test[ite,j] = (dataA[i,j] - minv[j]) / (maxv[j] - minv[j])
        y2_test[ite, int(dataA[i,M])] = 1
        ite += 1    
       
    Ntrv = int(Ntr/K)
    Ntrt = Ntr - Ntrv
    
    x2_train = []
    y2_train = []
    x2_val = []
    y2_val = []
    
    for k in range(K):
        x2_train.append(np.zeros((Ntrt,M),dtype='float32'))
        y2_train.append(np.zeros((Ntrt,C)))
        x2_val.append(np.zeros((Ntrv,M),dtype='float32'))
        y2_val.append(np.zeros((Ntrv,C)))
    
        itr = 0
        ite = 0
        for n in range(Ntr):
            i = orders[n]
            if n >= k*Ntrv and n < (k+1)*Ntrv:
                for j in range(M):
                    x2_val[k][ite,j] = (dataA[i,j] - minv[j]) / (maxv[j] - minv[j])
                y2_val[k][ite, int(dataA[i,M])] = 1
                ite += 1
            else:            
                for j in range(M):
                    x2_train[k][itr,j] = (dataA[i,j] - minv[j]) / (maxv[j] - minv[j])
                y2_train[k][itr,int(dataA[i,M])] = 1
                itr += 1
                
    return (x2_train,y2_train, x2_val, y2_val, x2_test, y2_test)

K = 10
(x_train,y_train,x_val,y_val, x_test,y_test) = gen_data_array_cv(K)
